# Quantum Error Correction

**Quantum error correction (QEC)** is used in quantum computing to protect quantum information from errors due to decoherence and other quantum noise. Quantum error correction is *essential* if one is to achieve fault-tolerant quantum computation that can deal not only with noise on stored quantum information, but also with faulty quantum gates, faulty quantum preparation, and faulty measurements.

(You can read more about it [here]( https://en.wikipedia.org/wiki/Quantum_error_correction).) 

Classical error correction employs redundancy. The simplest way to reliably store information is to store it multiple times and if in future due to an unknown error it conflicst - just take a majority vote. However copying quantum information is not possible due to the [*no-cloning theorem*](https://en.wikipedia.org/wiki/No-cloning_theorem). The no-cloning theorem states that there is no Unitary Operation $U$ such that $\forall |\phi\rangle$ $U|\psi\rangle|\phi\rangle = |\phi\rangle|\phi\rangle$. 
In simpler terms it in impossible to create independent copies of an unknown quantum state. 

However the central idea of classical error correction which is to encode few logical bits of data into greater number of physical bits is still applicable in Quantum Computing and is the cornerstone of Quantum Error Correction.

In this tutorial you will learn about:
* encoding one logical qubit into multiple physical qubits.
* guarding against a single bit flip error
* guarding against a single sign flip error
* implementing Shor's 9 qubit Error Correcting Code.

Let's go!

To begin, first prepare this notebook for execution (if you skip the first step, you'll get "Syntax does not match any known patterns" error when you try to execute Q# code in the next cells; if you skip the second step, you'll get "Invalid test name" error):

In [ ]:
%package Microsoft.Quantum.Katas::0.12.20070124

> The package versions in the output of the cell above should always match. If you are running the Notebooks locally and the versions do not match, please install the IQ# version that matches the version of the `Microsoft.Quantum.Katas` package.
> <details>
> <summary><u>How to install the right IQ# version</u></summary>
> For example, if the version of `Microsoft.Quantum.Katas` package above is 0.1.2.3, the installation steps are as follows:
>
> 1. Stop the kernel.
> 2. Uninstall the existing version of IQ#:
>        dotnet tool uninstall microsoft.quantum.iqsharp -g
> 3. Install the matching version:
>        dotnet tool install microsoft.quantum.iqsharp -g --version 0.1.2.3
> 4. Reinstall the kernel:
>        dotnet iqsharp install
> 5. Restart the Notebook.
> </details>


In [ ]:
%workspace reload

## The Basics:

The following section provides a brief Introduction of the Basics of Quantum Error Correction - the notation, basic ideas and error correction schemes.

## Encoding Logical Qubits into Physical Qubits

In classical computing, the bit can only be in the states $0$ and $1$. These states form the 2 logical states for classical computation. Error Correction involves duplicating these logical bits into multiple physical bits in order to make classical computing more fault-tolerant. In classical error correction $k$ physical bits can be used to represent $n$ logical bits where $k\geq n$. A famous example of such an encoding scheme is the [Hamming(7,4) Scheme](https://en.wikipedia.org/wiki/Hamming(7,4)) where 7 bits are used to encode 4 logical bits.

The *no cloning theorem* prevent the duplication of qubit. However we can store a logical qubit using multiple physical qubits. Henceforth we shall denote a logical qubit with the subscript $_L$. Thus the computational basis states of a single logical qubits would be denotes as $|0\rangle_L$ and $|1\rangle_L$. 

The **encoding scheme** is thus a *mapping* from logical qubits to physical qubits. 
$$E: |\psi_i\rangle_L \rightarrow |\phi_i\rangle$$ 
The mapping encoding $n$ logical qubits into $k$ physical qubits, is a linear transformation $T: \{0,1\}_L^n \rightarrow \{0,1\}^k$. It is a mapping from a subspace of dimension $2^n$ to a subspace of dimension $2^k$. 

This encoding is a quantum channel which is realised using $k-n$ ancillary qubits. The Unitary Operation is 
$$U_E: |\psi_i\rangle_L|0...0\rangle \rightarrow |\phi_i\rangle$$ or in vector space terms - $U_T: \{0,1\}_L^n\{0\}^{k-n} \rightarrow \{0,1\}^k$.

## Types of Error

In Classical Bits the only possible states are $0$ and $1$. Hence the only kind of error is a bit flip error. This error involves a bit randomly flipping: $0 \rightarrow 1$ or $1 \rightarrow 0$. Such an error can be modelled by randomly applying a $NOT$ gate to a bit. Classically these errors are often detected and corrected by introducing duplication of data and parity check bits. 

In Quantum Computing, the possible errors are not limited to bit flips, but also include sign flips (or phase flips). The most general error can be thought of as a Unitary $E$ applied to a qubit. 

A single qubit Unitary $U$ can be represented as a linear combination of the *Pauli Matrices* $\{\sigma_I,\sigma_X,\sigma_Y,\sigma_Z\}$. 

$$U = c_I\sigma_I + c_X\sigma_X + c_Y\sigma_Y + c_Z\sigma_Z$$

where the coefficients $c_I,c_X,c_Y,c_Z$ are complex numbers satisfying the following conditions:
$$|c_I|^2+|c_X|^2+|c_Y|^2+|c_Z|^2,$$
$$\mathcal{R}(c_Ic_X*) + \mathcal{I}(c_Yc_Z*) = 0,$$
$$\mathcal{R}(c_Ic_Y*) + \mathcal{I}(c_Xc_Z*) = 0,$$
$$\mathcal{R}(c_Ic_Z*) + \mathcal{I}(c_Xc_Y*) = 0,$$
where $\mathcal{R},\mathcal{I}$ are the Real and Imaginary Components respectively. 

$\sigma_Y$ can also be represnted in terms of $\sigma_X$ and $\sigma_Z$, since $\sigma_Y = i\sigma_X\sigma_Z$. Hence we can rewrite the Unitary $U$ as $c_I\sigma_I + c_X\sigma_X + c_Z\sigma_Z + c_{XZ}\sigma_X\sigma_Z$. A circuit which can correct all errors in the following group $\{I,X,Z,XZ\}$ can correct any single qubit error due to linearity.

## Part 1: Correcting Bit Flip Errors

The Bit Flip Error is one of the simplest errors. It involves the random application of $X$ gate. We assume that a $X$ gate may or may not be applied to a single physical qubit. To protect against this error we shall encode a single logical qubit $|\psi\rangle_L = \alpha|0\rangle_L + \beta|1\rangle_L$ into 3 physical qubits. We shall undo the encoding and in case of an error take the majority vote.

## <span style="color:blue">Task 1.1</span>:  Encoding Logical Qubit I

**Input:** A `logical` qubit in the state $|\psi\rangle = \alpha |0\rangle + \beta |1\rangle$ and two qubits in `ancillaryRegister`.

**Goal:** Create a state $|\psi\rangle := \alpha|000\rangle + \beta|111\rangle$ on these qubits.


In [ ]:
%kata T11_BitFlipEncoding_Test

operation BitFlipEncoding (logical: Qubit, ancillaryRegister : Qubit[]) : Unit {
    CNOT(logical, ancillaryRegister[0]);
    CNOT(logical, ancillaryRegister[1]);
}

## <span style="color:blue">Task 1.2</span>:  Correct Bit Flip Error

**Input:** An $X$ gate may be randomly applied to one of 3 qubits from Task 1.1. The resulting state is given as an input.

**Goal:** Transform `register[0]` into the state $|\psi\rangle = \alpha |0\rangle + \beta |1\rangle$. You may do anything you wish to the other 2 qubits.    


In [ ]:
%kata T12_CorrectBitFlip_Test

operation CorrectBitFlip (register : Qubit[]) : Unit {
    BitFlipEncoding(register[0], register[1..2]); // Repeat Task 1.1 to unentangle the qubits.
    CCNOT(register[1],register[2],register[0]); // Use CCNOT to fix the state of the 1st qubit.
}

## <span style="color:blue">Task 1.3</span>:  Detect Error Location

**Input:** An $X$ gate may be randomly applied to one of 3 qubits from Task 1.1. The resulting state is given as an input.

**Goal:** 
* Transform `register[0]` into the state $|\psi\rangle = \alpha |0\rangle + \beta |1\rangle$. 
* Find the location of the bit flip error i.e on which qubit - 1, 2 or 3 the $X$ gate was applied. If $X$ gate wasn't applied then return 0. 
* Reset the other 2 qubits to state $|00\rangle$.


In [ ]:
%kata T13_DetectErrorLocation_Test

open Microsoft.Quantum.Measurement;

operation DetectErrorLocation (register : Qubit[]) : Unit {
    CorrectBitFlip(register); // Correct 1st Qubit
    let bit2 = MResetZ(register[1])==One? true | false; // Reset to state 0
    let bit3 = MResetZ(register[2])==One? true | false; // Reset to state 1
    
    // Location Logic
    if (bit2 && bit3){
        return 1; 
    } else if (bit2 || bit3) {
        return bit2? 2 | 3;
    } else {
        return 0;
    }
    
}

## <span style="color:blue">Task 1.4</span>:  Detect and Fix a Bit Flip Error

**Input:** A `logical` qubit in the state $|\psi\rangle = \alpha |0\rangle + \beta |1\rangle$. An $`operation BitError (register: Qubit[]) :Unit` implementing $E_{bit}$ which denotes a possible Bit-Flip applied to any of the 3 qubits.  

**Goal:** Detect and Fix the Error which is applied by the `operation BitError(register: Qubit[]) :Unit`. 
At the end of the task the `logical` qubit should be in the same state as before. Additionally you must find out whether or not a bit flip occured and if it did - what was its location. 


In [ ]:
%kata T14_DetectFixBitFlipError_Test

operation DetectFixBitFlipError (logical: Qubit, BitError : (Qubit[] => Unit)) : Int {
    using(ancillaryRegister = Qubit[2]){
        BitFlipEncoding(logical,ancillaryQubit); // Encoding the Logical Qubits into Physical Qubits
        let register = [logical]+ancillaryQubit; 
        BitError(register); // Introducing Possible Bit Error in the register
        return DetectErrorLocation(register); // Fix the Logical Qubit, Detect Error Location and Reset Ancillary Qubit to 00
    }
}

<img src="Images/QEC_of_bit_flip_using_three_qubits.png" alt="QEC Circuit for Single Bit Flips" width="1500"/>


## Part 2: Correcting Sign Flip Errors

The Phase Flip Error is unique since it has no classical counterpart. It involves the random application of $Z$ gate. We assume that a $Z$ gate may or may not be applied to a single physical qubit. We already know how to correct bitflip errors. If we could transform $E_{phase}$ into $E_{bit}$, we could re-use the Bit Flip Error Correction Code to create a Sign Flip Error Correction Code.

## <span style="color:blue">Task 2.1</span>:  Convert Z into X

**Input:** A qubit in the state $|\psi\rangle = \alpha |0\rangle + \beta |1\rangle$.

**Goal:** Apply the $X$ gate on this qubit without using the [`X`](https://docs.microsoft.com/en-us/qsharp/api/qsharp/microsoft.quantum.intrinsic.x). 

<br>
<details>
    <summary><strong>Need a hint? Click here</strong></summary>
    Find $U$ such that $X$ = $U^{\dagger}ZU$. 
</details>

<br>
<details>
    <summary><strong>Need another hint? Click here</strong></summary>
    $Z|\pm\rangle = |\mp\rangle$ 
</details>

In [ ]:
%kata T21_ConvertZToX_Test

operation ConvertZToX (q: Qubit) : Unit {
    ApplyWith(H,Z,q); //Applies H†ZH on q
}

## <span style="color:blue">Task 2.2</span>:  Convert Phase Flip Error Into BitFlip Error

**Input:** An `operation PhaseError (register: Qubit[]) :Unit` implementing $E_{phase}$ which denotes a possible Phase-Flip applied to any of the 3 qubits.  

**Goal:** Return an `operation` which Convert the possible Phase-Flip applied in `operation PhaseError` into a Bit-Flip.

In [ ]:
%kata T22_ConvertPhaseFlipToBitFlipError_Test

operation ConvertPhaseFlipToBitFlipError (PhaseFlip : (Qubit[] => Unit)) : (Qubit[] => Unit) {
    let ConvertToHadamard = ApplyToEachA(H,_);// Convert Qubits to Hadamard Basis
    return ApplyWith(ConvertToHadamard, PhaseFlip, _);// A Phase Flip in Hadamard Basis is essentially a Bit-Flip
}

## <span style="color:blue">Task 2.3</span>:  Detect and Fix a Phase Flip Error

**Input:** A `logical` qubit in the state $|\psi\rangle = \alpha |0\rangle + \beta |1\rangle$. An $`operation PhaseError (register: Qubit[]) :Unit` which implements $E_{phase}$ denoting a possible Phase-Flip applied to any of the 3 qubits.  

**Goal:** Detect and Fix the Error which is applied by the `operation PhaseError(register: Qubit[]) :Unit`. 
At the end of the task the `logical` qubit should be in the same state as before. Additionally you must find out whether or not a bit flip occured and if it did - what was its location. 


In [ ]:
%kata T23_DetectFixPhaseFlipError_Test

operation DetectFixPhaseFlipError (logical: Qubit, PhaseError : (Qubit[] => Unit)) : Int {
    let BitError = ConvertPhaseFlipToBitFlipError(PhaseError); // Converted The PhaseError to BitError
    return DetectFixBitFlipError(logical, BitError); // Fixed the BitError
}

<img src="Images/QEC_of_phase_flip_using_three_qubits.png" alt="QEC Circuit for Single Sign Flips" width="1500"/>


## Part 3: Correcting Arbitrary One Qubit Errors



<img src="Images/QEC_Shors_Code.png" alt="QEC Circuit for Correcting Arbitary Single Qubit Error" width="1500"/>


## Conclusion

Congratulations on completing the Quantum Error Correction Kata! We hope you enjoyed this tutorial on quantum error correction ! If you're looking to learn more about quantum computing and Q# checkout other Katas.

The [Quantum Katas](https://github.com/microsoft/QuantumKatas/) are sets of programming exercises on quantum computing that can be solved using Q#. They cover a variety of topics, from the basics like the concepts of superposition and measurements to more interesting algorithms like Grover's search.